In [ ]:
%%capture
!pip install pyloudnorm
!pip install pysndfx
!pip install speechbrain
!pip install transformers
!pip install speechbrain.pretrained
!pip install torchmetrics

In [ ]:
import os
from multiprocessing import Process
import subprocess
import glob

import random
import warnings
import tarfile

import numpy as np
import pandas as pd
import pyloudnorm as pyln
import soundfile as sf
import tqdm as tqdm_module
import zipfile

import tqdm.contrib.concurrent
import functools
from pysndfx import AudioEffectsChain

import librosa

import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchaudio.transforms as T
import torchmetrics

from IPython.display import Audio

import speechbrain as sb
from speechbrain.dataio.dataio import read_audio
from IPython.display import Audio
from speechbrain.inference.separation import SepformerSeparation as separator

from torchmetrics.audio import ScaleInvariantSignalNoiseRatio
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio

In [ ]:
!unzip /content/drive/MyDrive/LibriMix.zip

In [ ]:
!unzip /content/drive/MyDrive/speechtask3.zip

In [ ]:
!pip install -r ../content/WSJ0Mix/extra_requirements.txt

In [ ]:
!python /content/WSJ0Mix/separation/train.py /content/WSJ0Mix/separation/hparams/sepformer.yaml --data_folder /content